# Metropolis Algorithm - Hopefield Network

Ismael CM
19/12/2022

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import random 

In [2]:
def read_patern(path='Patern/patern.txt'):
	patern = np.loadtxt(path, delimiter=',', dtype=np.uint8)
	return patern

In [3]:
einstein = np.loadtxt('Patern/einstein.txt', dtype=str)

In [4]:
patern = []
for i in range(len(einstein)):
	patern.append(list(einstein[i]))

In [5]:
patern = np.array(patern, dtype=np.uint8)

In [6]:
patern.shape

(100, 75)

In [7]:
def patern2point(patern):
	"""Convert a patern (matrix) to a list of points (x,y) where matrix[i][j] == 1
	
	x = j and y = -i + len(patern)

	Explanation:
		j selects the row (vertical movement) and i selects the column (horizontal movement)
		In a plot, x corresponds to the horizontal axis and y to the vertical axis

		Reversing i (i.e. -i) is necessary because the matrix is read from top to bottom. This is the same as reversing the y axis in a plot, so now everything is in the 4th quadrant.
		Adding len(patern) traslates everything to the 1st quadrant.
	"""
	x = []
	y = []
	for i in range(len(patern)): 			# i is the row
		for j in range(len(patern[i])): 	# j is the column
			if patern[i][j] == 1:
				x.append(j) 		    	
				y.append(-i + len(patern)) 
	return x,y

In [8]:
def interaction(net):
	a = np.average(net)
	N = len(net)
	M = len(net[0])
	w = np.zeros((N, M, N, M)) 		# All zeros 
	for i in range(N):
		for j in range(M):
			for k in range(N):
				for l in range(M):
					if (i,j) != (k,l): 	# so no need to do the case where i,j == k,l
						w[i][j][k][l] = (1/N**2) * (net[i][j] - a)*(net[k][l] - a)
						
	return w

In [9]:
def activation(net):
	N = len(net)
	M = len(net[0])
	w = interaction(net)
	theta = np.zeros((N, M))
	for i in range(N):
		for j in range(M):
			for k in range(N):
				for l in range(M):
					theta[i][j] += 0.5*w[i][j][k][l]
	return theta 

In [10]:
def difH(i, j, net, omega, theta):
	H = 0
	H_alt = 0
	N = len(net)
	M = len(net[0])
	for k in range(N):
		for l in range(M):
			if (k != i) and (l != j):
				H += omega[i][j][k][l]*net[k][l]

	H = (2*net[i][j]-1)*(H - theta[i][j]) 
	return H


In [11]:
def Hamilt(net, omega, theta):
	H = 0 
	N = len(net)
	for i in range(N):
		for j in range(N):
			H += theta[i][j]*net[i][j]
			for k in range(N):
				for l in range(N):
					H += -0.5*omega[i][j][k][l]*net[i][j]*net[k][l]
	return H

In [12]:
skull = read_patern("patern/skull.txt")
peng = read_patern("patern/peng.txt")
# x, y = patern2point(skull)
# plt.imshow(skull, cmap='binary')
# plt.scatter(x, y, s=50, c='k')
patern2 = read_patern("patern/panda.txt")

In [15]:
def active_phase(net, pMC, T):
	omega = interaction(net)
	theta = activation(net)
	N = len(net)
	M = len(net[0])

	s = np.random.randint(2, size=(N, M))

	pMC = pMC*N*M
	for k in range(pMC):
		if k%(N*M) == 0:
			# Plot
			plt.figure()
			plt.imshow(s, cmap='binary')
			plt.title(f"pMC = {int((k)/(N*N) + 1)}")
			plt.savefig(f"Plot/pMC{int((k)/(N*N) + 1)}.png")
			plt.close()
			print(f"Paso MoneteCarlo {int((k)/(N*N) + 1)}")

		aux_i = random.randint(0, N-1)
		aux_j = random.randint(0, N-1)
		
		H= difH(aux_i, aux_j, s, omega, theta)
		p = np.exp(-H/T, dtype=np.float64)
		if p > 1:
			s[aux_i][aux_j] = 1 - s[aux_i][aux_j]
		elif random.random() < p:
			s[aux_i][aux_j] = 1 - s[aux_i][aux_j]


In [18]:
active_phase(patern, 10, 1E-4)

In [ ]:
# # Active fase
# N = len(peng)
# # net = np.zeros((N, N), dtype=np.uint8)
# # net = peng.copy()
# net = np.random.randint(2, size=(N, N))
# pMC = 25*N*N
# T = 0.01
# for k in range(pMC):
# 	if k%(N*N) == 0:
# 		# Plot
# 		plt.figure()
# 		plt.imshow(net, cmap='binary')
# 		plt.title(f"pMC = {int((k)/(N*N) + 1)}")
# 		plt.savefig(f"Plot/pMC{int((k)/(N*N) + 1)}.png")
# 		plt.close()
# 		print(f"Paso MoneteCarlo {int((k)/(N*N) + 1)}")

# 	aux_i = random.randint(0, N-1)
# 	aux_j = random.randint(0, N-1)
	
# 	H, H_alt = difH(aux_i, aux_j, net, omega, theta)
# 	p = np.exp(-H/T)
# 	p_alt = np.exp(-H_alt/T)
# 	if random.random() < p:
# 		net[aux_i][aux_j] = 1 - net[aux_i][aux_j]